In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost
from sklearn.model_selection import RandomizedSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [78]:
train=pd.read_csv("Corona_NLP_train.csv",encoding='latin1') #reading train data csv file using pandas and assigning to train var

In [79]:
train.head() #accessing first few row of data from above csv

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [80]:
#function to remove the headers "UserName","ScreenName","Location","TweetAt", it removes all the columns of the header
def drop(p):
    p.drop(["UserName","ScreenName","Location","TweetAt"],axis=1,inplace=True)

In [81]:
drop(train) #calling drop function to drop headers from train

In [82]:
#getting first five element, we can see the headers "UserName","ScreenName","Location","TweetAt" are no longer in our train var
train.head() 


,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [83]:
#returns all the unique and "Sentiment" label's count from the csv
train["Sentiment"].value_counts()


Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [84]:
#function to replace the sentiment value with given value
#it will reeplace positive with 0, neg with 1 and so on.....
def rep(t):
        d={"Sentiment":{'Positive':0,'Negative':1,"Neutral":2,"Extremely Positive":3,"Extremely Negative":4}}
        t.replace(d,inplace=True)

In [85]:
rep(train) #calling rep function to replace the "Sentiment" or label with given number

In [86]:
train.head() #printing the first five element after calling rep function


,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,advice Talk to your neighbours family to excha...,0
2,Coronavirus Australia: Woolworths to give elde...,0
3,My food stock is not the only one which is emp...,0
4,"Me, ready to go at supermarket during the #COV...",4


In [87]:
#tokenizing using TweetTokenizer
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True)


In [88]:
#lemitenizing
lemmatizer=WordNetLemmatizer()


In [89]:
#stemming using  PorterStemmer
stemmer=PorterStemmer()


In [90]:
#Normalizing 
#removing all the symbols, commas, empty spaces.
#lowering all the letters and tokenizing and removing stopwords as well as lemitizing 
#adding normalized text to collect list
collect=[]
def preprocess(t):
    tee=re.sub('[^a-zA-Z]'," ",t)
    tee=tee.lower()
    res=tweettoken.tokenize(tee)
    for i in res:
        if i in stopwords.words('english'):
            res.remove(i)
    rest=[]
    for k in res:
        rest.append(lemmatizer.lemmatize(k))
    ret=" ".join(rest)
    collect.append(ret)

In [91]:
#calling preprocess function and applying for 41 document in csv train file
for j in range(41157):
    preprocess(train["OriginalTweet"].iloc[j])

In [92]:
#printing the 5th element of collect
collect[:5]

['menyrbie phil gahan chrisitv http co ifz fan pa http co xx ghgfzcc http co nlzdxno',
 'advice talk your neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med not order',
 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid outbreak http co binca vp p',
 'food stock not only one is empty please panic will enough food everyone not take than you need stay calm stay safe covid france covid covid coronavirus confinement confinementotal confinementgeneral http t co zrlg z j',
 'ready go supermarket covid outbreak because m paranoid because food stock litteraly empty the coronavirus a serious thing please panic cause shortage coronavirusfrance restezchezvous stayathome confinement http t co usmualq n']

In [93]:
#function to vectorizing the sentiment of the train data
def bow(ll):
    cv=CountVectorizer(max_features=200)
    x=cv.fit_transform(ll).toarray()
    return x

In [94]:
#calling bow function to vector the sentiment of collect dataframe
y=bow(collect)


In [95]:
#printing
y[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]], dtype=int64)

In [96]:
#getting the length
len(y[0][:])


200

In [97]:
#function to transform vector to array
def tfidf(xx):
    cv=TfidfVectorizer(max_features=4000)
    x=cv.fit_transform(xx).toarray()
    return x

In [98]:
#getting values of the train
values=train["Sentiment"].values


In [99]:
#printing values
values


array([2, 0, 0, ..., 0, 2, 1], dtype=int64)

In [101]:
#splitting the csv file into train and test
(x_train,x_test,y_train,y_test) = train_test_split(y,values, train_size=0.75, random_state=42)


In [102]:
#printing x_train
x_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [103]:
#applying decision trees
rnd_clf=RandomForestClassifier(n_estimators=200,random_state=42)


In [104]:
rnd_clf.fit(x_train,y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [105]:
rnd_clf.score(x_test,y_test) #calling applying decision trees function

0.4172983479105928

In [106]:
y_pred=rnd_clf.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
cm #printing cm

array([[1358,  592,  501,  338,  109],
       [ 702, 1014,  448,  106,  243],
       [ 477,  438,  913,   46,   45],
       [ 653,  183,  162,  617,   28],
       [ 287,  457,  148,   33,  392]], dtype=int64)

In [107]:
a=[400,500,600,700,800,900,1000]
for i in a:
    rnd_clf=RandomForestClassifier(n_estimators=i,random_state=42)
    rnd_clf.fit(x_train,y_train)
    t=rnd_clf.score(x_test,y_test)
    print(t)

0.4152575315840622
0.41438289601554906
0.41438289601554906
0.41243926141885323
0.41564625850340137
0.4182701652089407
0.41778425655976675


In [108]:
#naive base classifier 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB() #gives the highest probablity as output
clf.fit(x_train,y_train)

MultinomialNB()

In [109]:
clf.score(x_test,y_test) #prints out the accuracy with predicted of x_test and compare with y_test

0.3825072886297376

In [113]:
y=tfidf(collect) #transforming to array

In [114]:
(x_train,x_test,y_train,y_test) = train_test_split(y,values, train_size=0.75, random_state=42)


In [115]:
rnd_clf=RandomForestClassifier(n_estimators=200,max_leaf_nodes=8,random_state=42)
rnd_clf.fit(x_train,y_train)
rnd_clf.score(x_test,y_test)

0.2937803692905734

In [116]:
clf = MultinomialNB()
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.46763848396501456

In [ ]:
'''2. Explain what output does the *.score() function provide.
    score(x_test,y_test) function takes the matrix of x_test and predicts the value for x_test. Then it compares the value to y_test
    to measure the accuracy out of 1.


3. Explain why the score value from block #87 is higher than that from #34. Use the outputs from the two confusion matrices for this analysis. 
Include the above explanations as comments at the end of the notebook/program. 
= because in Out 87, MultinomialNB() algorithm is applied which gives out the highest probablity where as in #34 it gives lowest possible probablity'''